In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
mat1 = scipy.io.loadmat('data/d0404.mat')
TR=mat1['normd0404']
#TS=mat1['N_normNTS']
TR.shape

(1260, 8204)

In [2]:
sam_idx = np.linspace(0,8191, 2048, dtype='int8')
TR_sam=TR[:,sam_idx]
#TS_sam=TS[:,sam_idx]

#### DEFINE NETWROK

In [3]:
device_type = "/cpu:0"
with tf.device(device_type):
    n_input=8192
    n_output = 12


    weights = {
        'wc1' : tf.get_variable("wc_1", shape=[2, 7, 1, 1], initializer=tf.contrib.layers.xavier_initializer()),
        'wc2' : tf.get_variable("wc_2", shape=[2, 7, 1, 3], initializer=tf.contrib.layers.xavier_initializer()),
        'wc3' : tf.get_variable("wc_3", shape=[2, 7, 3, 3], initializer=tf.contrib.layers.xavier_initializer()),
        'wd1' : tf.get_variable("wc_4", shape=[(int)(4096/16)*1, 32], initializer=tf.contrib.layers.xavier_initializer()),
        'wd2' : tf.get_variable("wd_2", shape=[32, n_output], initializer=tf.contrib.layers.xavier_initializer())
    }

    biases = {
        'bc1' : tf.Variable(tf.random_normal([1], stddev=0.1)),
        'bc2' : tf.Variable(tf.random_normal([14], stddev=0.1)),
        'bc3' : tf.Variable(tf.random_normal([21], stddev=0.1)),
        'bd1' : tf.Variable(tf.random_normal([32], stddev=0.1)),
        'bd2' : tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }

In [5]:
device_type = "/cpu:0"
with tf.device(device_type):
    def conv_stap(_input, _w, _b, _keepratio):
        _input_r = tf.reshape(_input, shape=[-1, 2, 4096, 1])
        _pool0 = tf.nn.avg_pool(_input_r, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')
        _padded = tf.pad(_pool0, [[0,0],[0,0],[3,3],[0,0]], "CONSTANT")
        _conv1 = tf.nn.conv2d(_padded, _w['wc1'],strides=[1,1,1,1], padding='VALID')
        
        _mean, _var = tf.nn.moments(_conv1, [0,1,2])
        _conv1_n = tf.nn.batch_normalization(_conv1, _mean, _var, 0, 1, 0.0001)
        
        _conv1_r = tf.nn.sigmoid(_conv1)
        _pool1 = tf.nn.max_pool(_conv1_r, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')
        _dr1   = tf.nn.dropout(_pool1, _keepratio)

        _dense = tf.reshape(_dr1, [-1, _w['wd1'].get_shape().as_list()[0]])

        _fc1 = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
        _fc1_relu = tf.nn.sigmoid(_fc1)
        _fc_dr1 = tf.nn.dropout(_fc1_relu, _keepratio)

        _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])

        out = {
            'input_r': _input_r, 'conv1': _conv1, 'pool0': _pool0, 'pool1': _pool1
            ,'conv1_n':_conv1_n, 'conv1_r':_conv1_r, 'dense': _dense
            , 'dr1': _dr1, 'fc1': _fc1, 'fc1_relu':_fc1_relu, 'fc_dr1': _fc_dr1, 'out': _out
            , 'mean': _mean, 'var': _var
        }
        return out
    
def lrelu(x, alpha=0.1, max_value=None):
    negative_part=tf.nn.relu(-x)
    x=tf.nn.relu(x)
    
    if max_value is not None:
        x=tf.clip_by_value(x, tf.cast(0., dtype='float'), tf.cast(max_value, dtype='float'))
    
    x -= tf.constant(alpha, dtype='float') * negative_part
    return x


'''
_conv2 = tf.nn.conv2d(_dr1, _w['wc2'],strides=[1,1,1,1], padding='SAME')
_mean, _var = tf.nn.moments(_conv2, [0,1,2])
_conv2 = tf.nn.batch_normalization(_conv2, _mean, _var, 0, 1, 0.0001)
_conv2 = tf.nn.relu(_conv2)
_pool2 = tf.nn.max_pool(_conv2, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')
_dr2   = tf.nn.dropout(_pool2, _keepratio)

_conv3 = tf.nn.conv2d(_dr2, _w['wc3'],strides=[1,1,1,1], padding='SAME')
_mean, _var = tf.nn.moments(_conv3, [0,1,2])
_conv3 = tf.nn.batch_normalization(_conv3, _mean, _var, 0, 1, 0.0001)
_conv3 = tf.nn.relu(_conv3)
_pool3 = tf.nn.max_pool(_conv3, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')
_dr3   = tf.nn.dropout(_pool3, _keepratio)

'conv2': _conv2, 'pool2': _pool2, 'dense': _dense
, 'dr2': _dr2, 'conv3': _conv3, 'pool3': _pool3
, 'dr3': _dr3, 
'''

"\n_conv2 = tf.nn.conv2d(_dr1, _w['wc2'],strides=[1,1,1,1], padding='SAME')\n_mean, _var = tf.nn.moments(_conv2, [0,1,2])\n_conv2 = tf.nn.batch_normalization(_conv2, _mean, _var, 0, 1, 0.0001)\n_conv2 = tf.nn.relu(_conv2)\n_pool2 = tf.nn.max_pool(_conv2, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')\n_dr2   = tf.nn.dropout(_pool2, _keepratio)\n\n_conv3 = tf.nn.conv2d(_dr2, _w['wc3'],strides=[1,1,1,1], padding='SAME')\n_mean, _var = tf.nn.moments(_conv3, [0,1,2])\n_conv3 = tf.nn.batch_normalization(_conv3, _mean, _var, 0, 1, 0.0001)\n_conv3 = tf.nn.relu(_conv3)\n_pool3 = tf.nn.max_pool(_conv3, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')\n_dr3   = tf.nn.dropout(_pool3, _keepratio)\n\n'conv2': _conv2, 'pool2': _pool2, 'dense': _dense\n, 'dr2': _dr2, 'conv3': _conv3, 'pool3': _pool3\n, 'dr3': _dr3, \n"

# DEFINE FUNCTIONS

In [6]:

with tf.device(device_type):
    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_output])
    keepratio = tf.placeholder(tf.float32)

    pred = conv_stap(x, weights, biases, keepratio)['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    WEIGHT_DECAY_FACTOR = 0.0001
    l2_loss = tf.add_n([tf.nn.l2_loss(v)
                       for v in tf.trainable_variables()])
    cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
    optm = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1)) # count corrects
    accr = tf.reduce_mean(tf.cast(corr, tf.float32))
    init = tf.global_variables_initializer()


In [7]:
with tf.device(device_type):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    saver = tf.train.Saver()

# LEARNING

In [8]:
step = 105
idx1 = np.random.choice(105, size=20, replace=False)
idx2 = idx1+step
idx3 = idx1+step*2
idx4 = idx1+step*3
idx5 = idx1+step*4
idx6 = idx1+step*5
idx7 = idx1+step*6
idx8 = idx1+step*7
idx9 = idx1+step*8
idx10 = idx1+step*9
idx11 = idx1+step*10
idx12 = idx1+step*11
idx = np.concatenate([idx1,idx2,idx3,idx4,idx5,idx6,idx7,idx8,idx9,idx10,idx11,idx12])
idx


array([  45,   67,   18,   66,   12,   81,    4,   65,   27,   96,   70,
          2,   86,   13,   39,   16,   29,   10,   32,   73,  150,  172,
        123,  171,  117,  186,  109,  170,  132,  201,  175,  107,  191,
        118,  144,  121,  134,  115,  137,  178,  255,  277,  228,  276,
        222,  291,  214,  275,  237,  306,  280,  212,  296,  223,  249,
        226,  239,  220,  242,  283,  360,  382,  333,  381,  327,  396,
        319,  380,  342,  411,  385,  317,  401,  328,  354,  331,  344,
        325,  347,  388,  465,  487,  438,  486,  432,  501,  424,  485,
        447,  516,  490,  422,  506,  433,  459,  436,  449,  430,  452,
        493,  570,  592,  543,  591,  537,  606,  529,  590,  552,  621,
        595,  527,  611,  538,  564,  541,  554,  535,  557,  598,  675,
        697,  648,  696,  642,  711,  634,  695,  657,  726,  700,  632,
        716,  643,  669,  646,  659,  640,  662,  703,  780,  802,  753,
        801,  747,  816,  739,  800,  762,  831,  8

In [ ]:
with tf.device(device_type):
    training_epoch = 1000
    display_step = 100
    batch_size = 1260
       
    for epoch in range(training_epoch):
        avg_cost = 0.
        num_batch = int(1260/batch_size)
        for i in range(num_batch):
            randidx = np.random.randint(1260, size=batch_size)
            
            batch_xs = TR[randidx,0:8192]
            batch_ys = TR[randidx,8192:8204]

            #training
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio: 0.5})
            #Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})

        if epoch % display_step == 0 or epoch == training_epoch-1:

            print("Epoch: %03d/%03d cost: %.4f" % (epoch, training_epoch, avg_cost))
            training_acc = sess.run(accr, feed_dict={x: TR[idx,0:8192], y: TR[idx,8192:8204], keepratio:1.})
            test_acc     = sess.run(accr, feed_dict={x: TR[:,0:8192], y: TR[:,8192:8204], keepratio:1.})
            #acc = (training_acc*50 + test_acc*20 )/70
            print("TR acc : %.3f, TS acc : %.3f" % (training_acc, test_acc))
            #print("Test acc : %.3f" %(acc))


Epoch: 000/1000 cost: 2.7005
TR acc : 0.083, TS acc : 0.083
Epoch: 100/1000 cost: 2.4865
TR acc : 0.083, TS acc : 0.083
Epoch: 200/1000 cost: 2.4880
TR acc : 0.083, TS acc : 0.083
Epoch: 300/1000 cost: 2.4876
TR acc : 0.083, TS acc : 0.083
Epoch: 400/1000 cost: 2.4853
TR acc : 0.125, TS acc : 0.117
Epoch: 500/1000 cost: 2.4600
TR acc : 0.296, TS acc : 0.290


# STORE PARAMETERS

In [16]:
save_path = saver.save(sess, "parameters/model.ckpt")

# RESTORE PARAMETERS

In [36]:
saver.restore(sess, "parameters/model.ckpt")

# ACC CLASS BY CLASS

In [51]:
DS = np.concatenate([TR])
max_index = np.ndarray.argmax(DS[:,8192:],axis=1)
c01_index = np.where(max_index==0)
c02_index = np.where(max_index==1)
c03_index = np.where(max_index==2)
c04_index = np.where(max_index==3)
c05_index = np.where(max_index==4)
c06_index = np.where(max_index==5)
c07_index = np.where(max_index==6)
c08_index = np.where(max_index==7)
c09_index = np.where(max_index==8)
c10_index = np.where(max_index==9)
c11_index = np.where(max_index==10)
c12_index = np.where(max_index==11)
acc01 = sess.run(accr, feed_dict={x:DS[c01_index,0:8192][0,:,:], y:DS[c01_index,8192:8204][0,:,:], keepratio:1.})
acc02 = sess.run(accr, feed_dict={x:DS[c02_index,0:8192][0,:,:], y:DS[c02_index,8192:8204][0,:,:], keepratio:1.})
acc03 = sess.run(accr, feed_dict={x:DS[c03_index,0:8192][0,:,:], y:DS[c03_index,8192:8204][0,:,:], keepratio:1.})
acc04 = sess.run(accr, feed_dict={x:DS[c04_index,0:8192][0,:,:], y:DS[c04_index,8192:8204][0,:,:], keepratio:1.})
acc05 = sess.run(accr, feed_dict={x:DS[c05_index,0:8192][0,:,:], y:DS[c05_index,8192:8204][0,:,:], keepratio:1.})
acc06 = sess.run(accr, feed_dict={x:DS[c06_index,0:8192][0,:,:], y:DS[c06_index,8192:8204][0,:,:], keepratio:1.})
acc07 = sess.run(accr, feed_dict={x:DS[c07_index,0:8192][0,:,:], y:DS[c07_index,8192:8204][0,:,:], keepratio:1.})
acc08 = sess.run(accr, feed_dict={x:DS[c08_index,0:8192][0,:,:], y:DS[c08_index,8192:8204][0,:,:], keepratio:1.})
acc09 = sess.run(accr, feed_dict={x:DS[c09_index,0:8192][0,:,:], y:DS[c09_index,8192:8204][0,:,:], keepratio:1.})
acc10 = sess.run(accr, feed_dict={x:DS[c10_index,0:8192][0,:,:], y:DS[c10_index,8192:8204][0,:,:], keepratio:1.})
acc11 = sess.run(accr, feed_dict={x:DS[c11_index,0:8192][0,:,:], y:DS[c11_index,8192:8204][0,:,:], keepratio:1.})
acc12 = sess.run(accr, feed_dict={x:DS[c12_index,0:8192][0,:,:], y:DS[c12_index,8192:8204][0,:,:], keepratio:1.})
print("acc 01 : %.3f \nacc 02 : %.3f \nacc 03 : %.3f \nacc 04 : %.3f \nacc 05 : %.3f \nacc 06 : %.3f \nacc 07 : %.3f \nacc 08 : %.3f \nacc 09 : %.3f \nacc 10 : %.3f \nacc 11 : %.3f \nacc 12 : %.3f " 
      %(acc01, acc02, acc03, acc04, acc05, acc06, acc07, acc08, acc09, acc10, acc11, acc12))


acc 01 : 1.000 
acc 02 : 0.705 
acc 03 : 0.962 
acc 04 : 1.000 
acc 05 : 1.000 
acc 06 : 1.000 
acc 07 : 1.000 
acc 08 : 1.000 
acc 09 : 1.000 
acc 10 : 1.000 
acc 11 : 1.000 
acc 12 : 0.971 


# SAVE PARAMETERS FOR ANDROID

In [23]:
wc = sess.run(weights['wc1'])
wf1 = sess.run(weights['wd1'])
wf2 = sess.run(weights['wd2'])
bf1 = sess.run(biases['bd1'])
bf2 = sess.run(biases['bd2'])
Weights = {
    'wc':wc,
    'wf1':wf1,
    'wf2':wf2,
    'bf1':bf1,
    'bf2':bf2,
}
smat2 = scipy.io.savemat('data/savemat1.mat', Weights)
print(wc)
print(wc.shape)

[[[[ 1.1414156 ]]

  [[ 1.02894175]]

  [[ 1.22700822]]

  [[ 0.84509301]]

  [[-0.8457315 ]]

  [[-1.30150235]]

  [[-1.32011771]]]


 [[[-1.09077644]]

  [[-1.39412284]]

  [[-1.35836887]]

  [[-1.21993411]]

  [[-0.81074131]]

  [[-0.84921652]]

  [[ 1.03848755]]]]
(2, 7, 1, 1)


f = open("data/save1.txt", 'weights')
f.close()

# TEST

In [86]:
mat2 = scipy.io.loadmat('data/d0407.mat')
TRset = mat2['normd0407']
TRset.shape
TRset[0][1]

-0.0037235990721523624

In [87]:
t1 = sess.run(pred, feed_dict={x:TRset, keepratio:1.} )
max_index = np.ndarray.argmax(t1,axis=1)
max_index +1

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 4, 6, 5, 5, 5,
       1, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7])

In [41]:
pred1 = conv_stap(x, weights, biases, keepratio)['fc1_relu']

aa=sess.run(pred1, feed_dict={x:TRset[:1,:], keepratio:1.})
print(aa)
print(aa.shape)

[[ 0.          1.74668443  0.          0.          1.45507741  1.96513557
   1.53743565  0.          1.68994725  0.88912308  1.06961167  0.92114025
   0.76851153  0.          1.39314497  1.93270028  0.20780337  0.          0.
   1.52531314  0.          2.27788544  0.          0.0962585   0.          0.
   0.16577657  0.89982635  0.          0.          0.          0.        ]]
(1, 32)


In [42]:
aaa= tf.matmul(aa, wf2)
sess.run(aaa)

array([[ 11.15924263,  -1.19474912,  -5.00782442,  -7.39699841,
         -2.99421239,  -3.61619949,  -3.82316136, -12.86284161,
         -7.10066319, -10.6728363 ,  -8.37771416,  -1.03177774]], dtype=float32)